In [4]:
import psycopg2  # Para conectar ao PostgreSQL no RDS
from kafka import KafkaProducer
import json
import time
import random
from datetime import datetime

# Configurações do RDS (você já as tem no seu script)
DB_HOST = 'db-banco.c07gis2620tg.us-east-1.rds.amazonaws.com'
DB_PORT = '5432'
DB_NAME = 'db_banco'
DB_USER = 'postgres'
DB_PASSWORD = 'XqNuKeQw12345'

# Configurações do Kafka MSK (use os seus brokers)
bootstrap_servers = ["b-2.mskbancocluster.vwlqp6.c6.kafka.us-east-1.amazonaws.com:9094",
                     "b-3.mskbancocluster.vwlqp6.c6.kafka.us-east-1.amazonaws.com:9094",
                     "b-1.mskbancocluster.vwlqp6.c6.kafka.us-east-1.amazonaws.com:9094"]
topic_name = "MSKTopic"

def read_and_send_rds_data():
    conn = None  # Inicializa conn com None
    producer = None  # Inicializa producer com None
    try:
        # Conecta ao RDS
        conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USER, password=DB_PASSWORD)
        cur = conn.cursor()

        # Inicializa o produtor Kafka
        producer = KafkaProducer(bootstrap_servers=bootstrap_servers,
                                 value_serializer=lambda x: json.dumps(x).encode('utf-8'))

        while True:
            # Escolhe aleatoriamente uma tabela para ler (você pode ajustar isso)
            tables = ['clientes', 'transacoes', 'emprestimos', 'pagamentos']
            chosen_table = random.choice(tables)
            print(f"Lendo dados da tabela: {chosen_table}")

            # Executa a consulta SQL para obter os dados da tabela escolhida
            cur.execute(f"SELECT * FROM {chosen_table}")
            columns = [desc[0] for desc in cur.description]
            rows = cur.fetchall()

            for row in rows:
                # Cria um dicionário com os dados da linha
                data = dict(zip(columns, row))
                # Adiciona o nome da tabela para identificar a origem no Kafka (opcional)
                data['source_table'] = chosen_table
                # Envia a mensagem para o tópico Kafka
                producer.send(topic_name, value=data)
                print(f"Enviado para Kafka ({chosen_table}): {data}")

            producer.flush()  # Garante que todas as mensagens pendentes sejam enviadas
            time.sleep(10)  # Espera por 10 segundos antes de ler novamente (ajuste conforme necessário)

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
    finally:
        if conn:
            conn.close()
        if producer:
            producer.close()

if __name__ == "__main__":
    print("Iniciando a leitura de dados do RDS e envio para o Kafka...")
    read_and_send_rds_data()

Iniciando a leitura de dados do RDS e envio para o Kafka...
Ocorreu um erro: NoBrokersAvailable
